In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import dask.dataframe as dd
#import pyarrow.parquet as pq
from IPython.display import clear_output
import time
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, classification_report


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import autokeras as ak
from tensorflow.keras.utils import Sequence

KeyboardInterrupt: 

In [2]:
print(tf.__version__)
print(ak.__version__)

2.10.0
1.1.0


In [3]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


%%time
# Читаем и создаём тренировочный датасет
path_full_data = 'Dataset\\figma_plan\\full_gender_data.parquet'
path_train_data = 'public_train.pqt'

data_train = dd.read_parquet(path_full_data)
traget_data = dd.read_parquet(path_train_data).compute()

data_train = data_train.merge(traget_data[['user_id', 'is_male']], on = 'user_id', how = 'inner').compute()
data_train['is_male'] = data_train['is_male'].replace(to_replace=['NA'], value=[np.nan])
del traget_data

data_train = data_train.dropna(subset=['is_male'])
data_train.info(verbose=True, show_counts=True)
data_train.to_parquet('Dataset\\figma_plan\\data_train_gender.parquet', index=False)

In [30]:
%%time
data_train = dd.read_parquet('Dataset\\figma_plan\\data_train_gender.parquet').compute()
#data_train = data_train.drop(columns=['price'])
data_train['is_male'] = pd.to_numeric(data_train['is_male'], downcast = 'integer')
data_train.info(verbose=True, show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264326 entries, 0 to 264325
Data columns (total 3039 columns):
 #     Column                                            Non-Null Count   Dtype   
---    ------                                            --------------   -----   
 0     user_id                                           264326 non-null  int32   
 1     region_name_0                                     264326 non-null  category
 2     region_name_1                                     264326 non-null  category
 3     region_name_2                                     264326 non-null  category
 4     region_name_3                                     264326 non-null  category
 5     region_name_4                                     264326 non-null  category
 6     region_name_5                                     264326 non-null  category
 7     region_name_6                                     264326 non-null  category
 8     region_count                                      

In [31]:
print(data_train['price'].describe())
print(data_train['price'].mean())
print(data_train['price'].median())

count    257527.000000
mean      28667.298828
std       24237.466797
min          90.000000
25%       12010.000000
50%       18774.000000
75%       41071.000000
max      195657.000000
Name: price, dtype: float64
28667.299
18774.0


In [32]:
#data_train.loc[data_train['price'].isna(), 'price'] = data_train['price'].median()
data_train.loc[data_train['price'] < 1000 , 'price'] = 1000
#data_train['categrical_price'] = np.nan


In [ ]:
%%time
def func(row):
    if row['price'] <=12010:
        return 0
    elif 12010 < row['price'] <=41071:
        return 1
    elif row['price'] > 41071:
        return 2

data_train['categrical_price'] = data_train[['price']].apply(lambda row : func(row), axis=1)
print(data_train.info(verbose=True, show_counts=True))
data_train = pd.get_dummies(data_train, columns = ['categrical_price'])
print(data_train.info(verbose=True, show_counts=True))

In [33]:
target = data_train['is_male']#.to_numpy()
features = (data_train.drop(columns='is_male')).to_numpy()
col_name = (data_train.drop(columns=['is_male'])).columns
del data_train
print(col_name)

Index(['user_id', 'region_name_0', 'region_name_1', 'region_name_2',
       'region_name_3', 'region_name_4', 'region_name_5', 'region_name_6',
       'region_count', 'city_name_0',
       ...
       'bukvaprava.ru', 'belnovosti-by.turbopages.org',
       'ngs24-ru.turbopages.org', 'forum.mfd.ru', 'yomed.ru', '7info.ru',
       'other_url', 'other_female_urls_frac', 'other_male_urls_frac',
       'male_probability_by_urls'],
      dtype='object', length=3038)


In [13]:
target.value_counts()

1    135332
0    128994
Name: is_male, dtype: int64

In [14]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.33, random_state=12345)

In [15]:
print(features_train.shape)
print(target_train.shape)
target_train

(177098, 3038)
(177098,)


144177    0
239606    1
232874    1
2643      0
76492     1
         ..
158838    1
47873     1
86398     1
77285     0
217570    1
Name: is_male, Length: 177098, dtype: int8

In [ ]:
#target_train = target_train.to_numpy()


In [16]:
%%time
cat = CatBoostClassifier(task_type = 'GPU')
cat.fit(features_train,target_train, verbose=True)

Learning rate set to 0.025358
0:	learn: 0.6842688	total: 79.7ms	remaining: 1m 19s
1:	learn: 0.6759510	total: 153ms	remaining: 1m 16s
2:	learn: 0.6680655	total: 226ms	remaining: 1m 15s
3:	learn: 0.6606906	total: 290ms	remaining: 1m 12s
4:	learn: 0.6535441	total: 348ms	remaining: 1m 9s
5:	learn: 0.6468849	total: 406ms	remaining: 1m 7s
6:	learn: 0.6406505	total: 464ms	remaining: 1m 5s
7:	learn: 0.6346743	total: 522ms	remaining: 1m 4s
8:	learn: 0.6289956	total: 578ms	remaining: 1m 3s
9:	learn: 0.6237186	total: 632ms	remaining: 1m 2s
10:	learn: 0.6186148	total: 689ms	remaining: 1m 1s
11:	learn: 0.6137545	total: 746ms	remaining: 1m 1s
12:	learn: 0.6090480	total: 802ms	remaining: 1m
13:	learn: 0.6046061	total: 859ms	remaining: 1m
14:	learn: 0.6005129	total: 915ms	remaining: 1m
15:	learn: 0.5964571	total: 972ms	remaining: 59.8s
16:	learn: 0.5927057	total: 1.03s	remaining: 59.4s
17:	learn: 0.5891159	total: 1.08s	remaining: 59.1s
18:	learn: 0.5857655	total: 1.14s	remaining: 58.9s
19:	learn: 0.58

In [17]:
print(f'GINI по полу трейн {2 * roc_auc_score(target_train, cat.predict_proba(features_train)[:,1]) - 1:2.3f}')
print(f'GINI по полу тест {2 * roc_auc_score(target_valid, cat.predict_proba(features_valid)[:,1]) - 1:2.3f}')

GINI по полу трейн 0.762
GINI по полу тест 0.744


In [45]:
cat_imp = pd.Series(cat.get_feature_importance(), index=col_name).sort_values(ascending=False)
list(cat_imp[cat_imp>0].index)

['male_probability_by_urls',
 'other_female_urls_frac',
 'other_male_urls_frac',
 'syndication.realsrv.com',
 'baza.drom.ru',
 'apple.com',
 'i.pinimg.com',
 'city_count',
 'i.ytimg.com',
 'l.instagram.com',
 's-cdn.sportbox.ru',
 'rpc-php.trafficfactory.biz',
 'drive2-ru.turbopages.org',
 'cpe_manufacturer_name_3',
 'drive2.ru',
 'irecommend.ru',
 'online.sberbank.ru',
 'instagram.com',
 'day_fraction',
 'yandex.ru',
 'yastatic.net',
 'news.gnezdo.ru',
 'price',
 'pozdravok.ru',
 'zen.yandex.ru',
 'auto.ru',
 'apteka.ru',
 'xn--90adear.xn--p1ai',
 'forum.ixbt.com',
 'povar-ru.turbopages.org',
 'm.ok.ru',
 'g0.sunlight.net',
 '7days.ru',
 'o2.mail.ru',
 'avatars.mds.yandex.net',
 'ru.wargaming.net',
 'ads.betweendigital.com',
 'm.sports.ru',
 'st.mycdn.me',
 'loveome.club',
 'razdvabm.com',
 'prodoctorov.ru',
 'mag.auto.ru',
 'sdk.meitusdk.com',
 'discord.com',
 'wildberries.ru',
 't.me',
 'mail.yandex.ru',
 'getcourse.ru',
 'blitz.plus',
 'static.1tv.ru',
 'esia.gosuslugi.ru',
 'googl

### Без обработки

GINI по полу трейн 0.762
GINI по полу тест 0.744

###


In [59]:
submit = dd.read_parquet('submit_2.pqt').compute()
submit = submit.sort_values('user_id', ignore_index=True)

submit['age'] = pd.Series(model.predict(test))
submit['is_male'] = pd.Series(model.predict_proba(features_valid)[:,1])
submit.to_csv('submit_IIntegration.csv', index=False)

submit

,user_id
0,6
1,7
2,9
3,10
4,11
...,...
144719,415306
144720,415310
144721,415314
144722,415315
